<a href="https://colab.research.google.com/github/kelliebanzon/slo-hacks-2019/blob/master/playground.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Setup**

In [0]:
!pip install spotipy

  Stored in directory: /root/.cache/pip/wheels/76/28/19/a86ca9bb0e32dbd4a4f580870250f5aeef852870578e0427e6
Successfully built spotipy


In [0]:
import sys
import json
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import numpy as np
import glob


In [0]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

In [0]:
file_path = glob.glob("/gdrive/My Drive/programming/hackathons/sloHacks2019/tracks.csv")
tracks = pd.read_csv("/gdrive/My Drive/programming/hackathons/sloHacks2019/tracks.csv")

In [5]:
tracks.head()

,rating,track_1,artist_1,album_1,uri_1,track_2,artist_2,album_2,uri_2,is_pop
0,0.0,Palm Dreams,Hayley Kiyoko,Expectations,spotify:track:3qDaH8tgcdzdpxaL7CyW2s,The 1975,The 1975,"I like it when you sleep, for you are so beaut...",spotify:track:4ZEvWpjUBtL9gjVIzB2Wbr,NaN
1,0.0,Let It Be,Hayley Kiyoko,Expectations,spotify:track:181uk7o0ajEG8F1HYrqbxu,The 1975,The 1975,A Brief Inquiry Into Online Relationships,spotify:track:7dvM0LbJ4pu1tDJnCH1Ahg,NaN
2,0.0,TOOTIMETOOTIMETOOTIME,The 1976,A Brief Inquiry Into Online Relationships,spotify:track:33DzKnwuBE6lfOiADwzd5E,How To Draw / Petrichor,The 1976,A Brief Inquiry Into Online Relationships,spotify:track:1bTfkhyNrSmthxL7P5yGDJ,NaN
3,0.0,Vibe,Tove Stryke,NaN,spotify:track:7rXOrI115JVyhBfM2DRKwZ,Rooftop - Acoustic,Clara Mae,Sorry for Writing All the Songs About You,spotify:track:7m34nMTiU0KwhsMSBmUhde,NaN
4,0.0,Good Mourning,Halsey,hopeless fountain kingdom,spotify:track:6Ck8Lq4srTWQP1PXZ9P8aZ,Wait Up,Charlotte Lawrence,NaN,spotify:track:6VM9H9tmexHFNLePOXOVPX,NaN


**Authentication**

In [0]:
username = "USERNAME"
scope = "user-library-read"

client_credentials_manager = SpotifyClientCredentials(client_id='CLIENT-ID', client_secret='CLIENT-SECRET')
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

**Get Audio Features**

In [0]:
# returns dataframe with the 'Audio Features' for pair of songs in a single row
def getAudioFeatures(row):
  uri_1 = row['uri_1']
  uri_2 = row['uri_2']
 
  raw_1  = sp.audio_features(uri_1)
  raw_2  = sp.audio_features(uri_2)
  
  features_1 = pd.DataFrame(raw_1)
  features_2 = pd.DataFrame(raw_2)
  features_1 = features_1.drop(columns=['duration_ms', 'key', 'id', 'uri', 'track_href', 'analysis_url'])
  features_2 = features_2.drop(columns=['duration_ms', 'key', 'id', 'uri', 'track_href', 'analysis_url'])
  
  features_1 = features_1.add_suffix("_1")
  features_2 = features_2.add_suffix("_2")
 
  return pd.concat([features_1, features_2], axis=1)

In [15]:
# loop though handmade list containing pairs of songs
features = getAudioFeatures(tracks.loc[0])
for i in range(1, len(tracks)):
  features = pd.concat([features, getAudioFeatures(tracks.loc[i])], axis=0)

retrying ...3secs
retrying ...1secs
retrying ...2secs
retrying ...1secs


In [17]:
features.head()

,acousticness_1,danceability_1,energy_1,instrumentalness_1,liveness_1,loudness_1,mode_1,speechiness_1,tempo_1,time_signature_1,...,energy_2,instrumentalness_2,liveness_2,loudness_2,mode_2,speechiness_2,tempo_2,time_signature_2,type_2,valence_2
0,0.122,0.654,0.772,0.000063,0.2190,-5.169,1,0.0589,103.999,4,...,0.459,0.022300,0.4100,-9.873,1,0.0448,85.108,3,audio_features,0.0726
0,0.395,0.588,0.834,0.000004,0.1270,-4.367,1,0.0466,79.959,4,...,0.129,0.000000,0.0918,-13.173,1,0.1030,138.404,4,audio_features,0.2780
0,0.209,0.667,0.798,0.015000,0.0972,-5.935,1,0.0675,114.981,4,...,0.792,0.292000,0.0727,-7.951,1,0.2610,138.228,4,audio_features,0.0386
0,0.297,0.586,0.567,0.000000,0.1010,-8.832,1,0.0522,132.027,4,...,0.246,0.000000,0.4110,-12.258,1,0.1570,178.577,4,audio_features,0.5120
0,0.713,0.629,0.530,0.027400,0.4120,-9.943,0,0.0556,99.806,4,...,0.705,0.000032,0.1280,-2.941,0,0.0344,101.075,4,audio_features,0.3820


In [0]:
# NOTE: run only unpdating dataset
# features.to_csv("tags.csv")